In [1]:
from simpletransformers.question_answering import QuestionAnsweringModel
from nested_lookup import nested_lookup
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download("punkt")
import string
import shutil
import datetime as dt
import torch

[nltk_data] Downloading package stopwords to /home/ilker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ilker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('train-v0.1.json' , encoding='utf-8') as json_file: 
    data = json.load(json_file)
    
data_list = list()
for i in data['data']:
    for j in i['paragraphs']:
        for k in j['qas']:
            k['is_impossible'] = False
            for z in k['answers']:
                tmp = z['answer_start']
                z['answer_start'] = int(tmp)
        data_list.append(j)

## training

In [3]:
def get_model(model_name):
    model_base = model_name.split("/")[1].split("-")[0]
    model = QuestionAnsweringModel(model_base, model_name, use_cuda = True)
    return model

## make predictions

In [4]:
def soru_sor(soru,id, context,model):
    soru = [{
        'context': context,

        'qas': [
            {
                'question':soru,
                'id':id 
            }
        ]
    }]
    
    return model.predict(soru)

In [5]:
def test_data_input(input_file):
    # Context into list
    f = open(input_file,"r",encoding = "utf-8")
    first_line = f.readline()
    del first_line
    # context
    context_list =    []
    context_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("context"):
            context_line = True
            a_list = []
        elif line.startswith("topic"):
            lines = ' '.join(a_list)
            context_list.append(lines)
            context_line = False
        if context_line:
            a_list.append(line.rstrip("\n").lstrip("\ufeff").strip("|").lstrip("context:").lstrip(": ").lstrip(" "))
    # questions into list
    f = open(input_file,"r",encoding = "utf-8")
    question_list =    []
    question_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("question"):
            question_line = True
            a = []
        elif line.startswith("answer"):
            lines = ' '.join(a)    
            question_list.append(lines)
            question_line = False
        else:continue
        if question_line:
            a.append(line.rstrip("\n").lstrip("\ufeff").strip("|").lstrip("question:").lstrip(" ").rstrip(" "))
    # id into list
    f = open(input_file,"r",encoding = "utf-8")
    id_list =    []
    id_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("id"):
            id_line = True
        else:continue
        if id_line:
            id_list.append(line.rstrip("\n").lstrip("id:").strip("|").strip(" "))

    # To find number of questions for each topic
    f = open(input_file,"r",encoding = "utf-8")
    topic = []
    topic_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("topic"):
            topic_line = True
            a = []
        elif line.startswith("question"):
            lines = ' '.join(a)
            topic.append(lines)
            topic_line = False
        else:continue
        if topic_line:
            a.append(line.rstrip("\n").lstrip("topic:").rstrip("|").rstrip(" ").strip(" "))
    # answers
    f = open(input_file,"r",encoding = "utf-8")
    answer_list =    []
    answer_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("answer"):
            answer_line = True
            a = []
        elif line.startswith("id"):
            lines = ' '.join(a)    
            answer_list.append(lines)
            answer_line = False
        else:continue
        if answer_line:
            a.append(line.rstrip("\n").strip("|").lstrip("answer:").lstrip(" ").rstrip(" "))    
    # make contexts as much as multiplied by their questions
    topic_counter_dict = {}
    topic_counts = []
    for top in topic:
        if not top in topic_counter_dict.keys():
            topic_counter_dict[top] = 1
        elif top in topic_counter_dict.keys():
            topic_counter_dict[top] += 1
    for key,value in topic_counter_dict.items():
        topic_counts.append(value)
    a = [[con]*topic_counts[i] for i,con in enumerate(context_list)]
    context_list_with_num = []
    for context_list in a:
        for context_repeat in context_list:
            context_list_with_num.append(context_repeat)
    return context_list_with_num, question_list, id_list, topic, answer_list

In [6]:
def remove_stopwords(some_list):
    for token in some_list:
        if token in stopwords.words("turkish"):
            del some_list[some_list.index(token)]
    return some_list

In [7]:
def exact_match(predicted, actual):
    matched_answers = []
    guessed_true = 0
    for i, answer in enumerate(actual):
        if predicted[i].lower().translate(str.maketrans('', '', string.punctuation)).strip(" ") == actual[i].lower().translate(str.maketrans('', '', string.punctuation)).strip(" "):
            guessed_true += 1 
            matched_answers.append(predicted[i])
    return guessed_true,matched_answers

# prepare data

In [8]:
with open('train-v0.1.json' , encoding='utf-8') as json_file: 
    data = json.load(json_file)
    
data_list = list()
for i in data['data']:
    for j in i['paragraphs']:
        for k in j['qas']:
            k['is_impossible'] = False
            for z in k['answers']:
                tmp = z['answer_start']
                z['answer_start'] = int(tmp)
        data_list.append(j)

In [10]:
len(data_list)

2232

In [11]:
data_list[0]

{'qas': [{'question': 'el-Hasan b. Muhammed el-Vezzan isimli bilgin avrupa’da nasıl tanınmaktadır ?',
   'id': 1,
   'answers': [{'answer_start': 171, 'text': 'Afrikalı Leo'}],
   'is_impossible': False},
  {'question': 'el-Hasan b. Muhammed el-Vezzan hangi şehirde büyümüştür ?',
   'id': 2,
   'answers': [{'answer_start': 278, 'text': 'Fas (Fez) şehrinde'}],
   'is_impossible': False},
  {'question': 'el-Hasan b. Muhammed el-Vezzan sicilyalı korsanların eline düştüğünde nerelere satılmıştır ?',
   'id': 3,
   'answers': [{'answer_start': 587,
     'text': 'ilk olarak Napoli’ye daha sonra Roma’ya'}],
   'is_impossible': False},
  {'question': 'el-Hasan b. Muhammed el-Vezzan kim tarafından vaftiz edilmiştir ?',
   'id': 4,
   'answers': [{'answer_start': 635, 'text': 'Papa X. Leo'}],
   'is_impossible': False},
  {'question': 'el-Hasan b. Muhammed el-Vezzan kaç yılında vaftiz edilmiştir ?',
   'id': 5,
   'answers': [{'answer_start': 658, 'text': '6.1.1520 yılında'}],
   'is_impossible'

In [11]:
context_list_with_num, question_list, id_list, topic, answer_list = test_data_input("test_questions_compiled.txt")

# models

In [12]:
### function for the whole process
def find_exact_match_number(model_name):
    
    t1 = dt.datetime.now()
    
    """
    istediginiz bir şey varsa (predictions, tokenized_preds gibi) return ettirebilirsiniz
    en başta cache_dir, outputs, runs klasörlerini sildiriyorum. çünkü bir model oluşturunca
    bunları da oluşturuyor ve bu klasörleri silmeden yeni model çağıramıyor
    %% capture her şeyi bastırmaması için (verbose = 0 gibi)
    """
    dirs = !ls
    dirs = pd.Series(dirs)
    dels = dirs[dirs.isin(["cache_dir","outputs","runs"])].to_list()

    for folder_name in dels:
        try:
            shutil.rmtree(folder_name)
        except:
            pass
        
    model = get_model(pretrained_models[0])
    model.train_model(data_list)
    context_list_with_num, question_list, id_list, topic, answer_list = test_data_input("test_questions_compiled.txt")
    
    predicted_answers = []

    for i in range(len(context_list_with_num)):
        answerNprobs = (soru_sor(question_list[i], id_list[i], context_list_with_num[i], model))
        answer = answerNprobs[0][0]["answer"][0]
        predicted_answers.append(answer)
    
    tokenized_preds = [word_tokenize(sentence) for sentence in predicted_answers]
    tokenized_answers = [word_tokenize(sentence) for sentence in answer_list]
    predictions = [" ".join(remove_stopwords(token_list)) for token_list in tokenized_preds]
    actual_answers = [" ".join(remove_stopwords(token_list)) for token_list in tokenized_answers]
    exact_match_number, exact_match_answers = exact_match(predictions, actual_answers)
    
    t2 = dt.datetime.now()
    
    torch.cuda.empty_cache()
    
    return exact_match_number, t1, t2, predicted_answers, answer_list

def print_results(model_name, exact_match_number, t1, t2):
    
    print(model_name)
    print("Exact_match_number:{}".format(exact_match_number))
    print("Training + prediction took: " + str(t2-t1).split(".")[0] + " (format: hh:mm:ss)")
    print(" ") 

# all models

In [2]:
pretrained_models = ["dbmdz/bert-base-turkish-128k-cased",
"dbmdz/bert-base-turkish-128k-uncased",
"dbmdz/bert-base-turkish-cased",
"dbmdz/bert-base-turkish-uncased",
"dbmdz/distilbert-base-turkish-cased",
"dbmdz/electra-base-turkish-cased-discriminator",
"dbmdz/electra-small-turkish-cased-discriminator"]

In [14]:
%%capture
model_name = pretrained_models[0]
exact_match_number, t1, t2, predicted_answers, answer_list = find_exact_match_number(model_name)

In [26]:
res = pd.DataFrame(predicted_answers + [str(t2-t1).split(".")[0]] , columns = ["predicted"])
res["true"] = answer_list + [model_name]

In [27]:
res.tail()

,predicted,true
105,Gazeteci-Yazar Kaya Muzaffer Ilıcak,Gazeteci-Yazar Kaya Muzaffer Ilıcak
106,Gazeteci-Yazar Kaya Muzaffer Ilıcak,Cihat Uskan
107,,Shotokan
108,,Türkiye Üniversite Sporları Federasyonu
109,0:03:48,dbmdz/bert-base-turkish-128k-cased


In [33]:
"results/" + (model_name + "_default").replace("/", "_")

'results/dbmdz_bert-base-turkish-128k-cased_default'

In [35]:
res.to_csv("results/" + (model_name + "_default.csv").replace("/", "_"), index = None)

In [3]:
pretrained_models[0] in pretrained_models

True